In [1]:
%run input/Format.ipynb
import ROOT as root
from array import array
import math as math
root.gErrorIgnoreLevel = root.kFatal
import multiprocessing as mp
root.gStyle.SetOptFit(1111)
%jsroot on
%rm output/deltas/*.png

/home/yoren/.local/lib/python3.10/site-packages/nbformat/__init__.py:96: MissingIDFieldWarning: Cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.
  validate(nb)


Welcome to JupyROOT 6.30/06


In [2]:
iOption0 = 0
sim = 1
Min_Entries = 10
N_files = 1
Do_save = 1

In [3]:
N_hists = 6
colors=[1,2,4,root.kGreen+2,root.kMagenta,root.kOrange+4,root.kGray,root.kCyan,root.kTeal]
file_path="input/"
file_names="my-m_ee_Run14AuAu_86th_new_19862_755runs" 
#m_ee_Run14AuAu_84th_new_19850_337runs m_ee_Run14AuAu_83rd_new_19842_1065runs m_ee_Run14AuAu_85th_new_19858_120runs
hist_select_3D_names = ["DCA12_hist_0","DCA12_hist_1","DCA12_hist_2","DCA12_hist_3","DCA12_hist_4","hist_dca_x","hist_dca_y"]
N_hists = len(hist_select_3D_names)

In [4]:
if sim :
    file_path="../sim/output/Newembed/dca/" 
    file_names="piminus_50M_new_v7"#piminus_50M_v7 piminus_50M_v8 piminus_50M_v9
    N_hists = N_hists - 2

In [5]:
hists_read = []

infile = root.TFile.Open(file_path+file_names+f".root", "read")
for ihist in range(N_hists):
    hists_read.append(infile.Get(hist_select_3D_names[ihist]))
    hists_read[-1].SetDirectory(root.nullptr)
infile.Close()

In [6]:
z_edges = [-12+3*i for i in range(9)]
z_means = [-10.5+3*i for i in range(8)]
z_edges = array('d', z_edges)
z_means = array('d', z_means)
phi_edges = [-1.5 + 0.30*i for i in range(21)]
phi_means = [-1.35 + 0.30*i for i in range(20)]
phi_edges = array('d', phi_edges)
phi_means = array('d', phi_means)
def get_phi_zed_bin(in_value):
    if in_value > 179: in_value -= 180
    zed = int(in_value / 20)
    phi = int((in_value - zed * 20) )
    return zed, phi
def get_phi_value(phi_bin):
    return phi_means[phi_bin] if phi_bin >= 0 and phi_bin < len(phi_means) else 0
def get_zed_value(zed_bin):
    return z_means[zed_bin] if zed_bin >= 0 and zed_bin < len(z_means) else 0

In [7]:
def copycat_hist(hist,seed=0):
    seed += root.gRandom.Integer(1000)
    new_hist = root.TH3D(hist.GetName()+f"{seed}", hist.GetTitle()+f"{seed}", hist.GetNbinsX(), hist.GetXaxis().GetBinLowEdge(1), hist.GetXaxis().GetBinUpEdge(hist.GetNbinsX()),\
        hist.GetNbinsY(), hist.GetYaxis().GetBinLowEdge(1), hist.GetYaxis().GetBinUpEdge(hist.GetNbinsY()),\
        hist.GetNbinsZ(), hist.GetZaxis().GetBinLowEdge(1), hist.GetZaxis().GetBinUpEdge(hist.GetNbinsZ()))
    for ix in range(1, hist.GetNbinsX()+1):
        for iy in range(1, hist.GetNbinsY()+1):
            for iz in range(1, hist.GetNbinsZ()+1):
                new_hist.SetBinContent(ix, iy, iz, hist.GetBinContent(ix, iy, iz))
    return new_hist

In [8]:
ihits = 1
in_hist = hists_read[ihits]
c1 = root.TCanvas("c1", "c1", 1600, 1000)
c1.Divide(3,2)
local_bin_endges = [[1,100],[21,30],[51,60],[25,25],[55,55]]
sigmas = []
for i in range(5):
    if sim: continue
    c1.cd(i+1)
    hist_proj = in_hist.ProjectionX(in_hist.GetName()+f"hist_proj{i}", 66, 101, local_bin_endges[i][0], local_bin_endges[i][1])
    hist_proj.Draw("")
    hist_proj1 = in_hist.ProjectionX(in_hist.GetName()+f"hist_proj{i}_1", 1, 35, local_bin_endges[i][0], local_bin_endges[i][1])
    hist_proj1.Draw("same")
    hist_proj.Fit("gaus", "Q","",-150,150)
    hist_proj1.Fit("gaus", "Q","",-150,150)
    sigmas.append([hist_proj.GetFunction("gaus").GetParameter(2), hist_proj1.GetFunction("gaus").GetParameter(2)])

c1.cd(6)
hist_projy = in_hist.ProjectionY(in_hist.GetName()+"hist_projy", 21, 30, 1, 160)
hist_projy.Draw("")

c1.Draw()

In [9]:
print("Sigma values for the histograms:")
for i, sigma in enumerate(sigmas):
    print(f"Histogram {i+1}: {sigma[0]:.3f}, {sigma[1]:.3f}")

Sigma values for the histograms:


In [10]:
ihits = 1
pt = 10
in_hist = copycat_hist(hists_read[ihits])
for ibinx in range(in_hist.GetNbinsX()):
    for ibiny in range(in_hist.GetNbinsY()):
        for ibinz in range(in_hist.GetNbinsZ()):
            if ibiny<50+pt and ibiny>50-pt:
                in_hist.SetBinContent(ibinx, ibiny, ibinz, 0)
hist_proj = in_hist.Project3D("xz")

c1 = root.TCanvas("c1", "c1", 800, 600)
profile_x = hist_proj.ProfileX(hist_proj.GetName()+"profileX")
profile_x.Rebin(1)
#profile_x.Fit(fit_func, "Q", "", 50, 60)
profile_x.Draw("")
c1.Draw()


In [11]:
if False :
    x_offsets = array("d")
    x_abscissas = array("d")
    x_offsets_errors = array("d")
    x_arrays = {}
    for ibinx in range(profile_x.GetNbinsX()):
        x_offsets.append(profile_x.GetBinContent(ibinx+1))
        x_offsets_errors.append(profile_x.GetBinError(ibinx+1))
        x_abscissas.append(profile_x.GetBinCenter(ibinx+1))
    for i, (xx, yy, zz) in enumerate(zip(x_abscissas, x_offsets, x_offsets_errors)):
        z_bin, phi_bin = get_phi_zed_bin(xx)
        phi = get_phi_value(phi_bin)
        if z_bin not in x_arrays:
            x_arrays[z_bin] = [array('d'),array('d'),array('d'),array('d')]
        x_arrays[z_bin][0].append((yy))
        x_arrays[z_bin][1].append((phi))
        x_arrays[z_bin][2].append((zz))
        x_arrays[z_bin][3].append(0.15)
    
    print("x_arrays", x_arrays)

In [12]:
if True :
    x_offsets = array("d")
    x_abscissas = array("d")
    x_offsets_errors = array("d")
    x_arrays = {}
    all_projections = []
    for ibinx in range(profile_x.GetNbinsX()):
        local_mean = in_hist.GetBinContent(ibinx+1)
        local_projection = in_hist.ProjectionX(in_hist.GetName()+f"hist_proj_z_{ibinx}", 1, 101, ibinx+1, ibinx+1)
        local_projection.Fit("gaus", "Q","",local_mean-120,local_mean+120)
        all_projections.append(local_projection)
        if local_projection.GetEntries() > 500 and local_projection.GetFunction("gaus"):
            c1 = root.TCanvas("c1", "c1", 800, 600)
            x_offsets.append(local_projection.GetFunction("gaus").GetParameter(1))
            x_offsets_errors.append(2000./local_projection.GetFunction("gaus").Integral(-120, 120) **0.5 )
            local_projection.Draw("")
            c1.SaveAs(f"output/deltas/fit_zed_{ibinx}.png")
        else:
            x_offsets.append(0)
            x_offsets_errors.append(0)
        x_abscissas.append(profile_x.GetBinCenter(ibinx+1))
    for i, (xx, yy, zz) in enumerate(zip(x_abscissas, x_offsets, x_offsets_errors)):
        z_bin, phi_bin = get_phi_zed_bin(xx)
        phi = get_phi_value(phi_bin)
        if z_bin not in x_arrays:
            x_arrays[z_bin] = [array('d'),array('d'),array('d'),array('d')]
        x_arrays[z_bin][0].append((yy))
        x_arrays[z_bin][1].append((phi))
        x_arrays[z_bin][2].append((zz))
        x_arrays[z_bin][3].append(0.15)
    
    print("x_arrays", x_arrays)

x_arrays {0: [array('d', [0.0, 0.0, 0.0, 0.0, -2.7850536715704544, 0.0, 21.110019906500707, 0.0, 8.800463037208873, 0.0, 0.0, 0.0, 0.0, 13.132174061796619, 6.999574930505389, 1.6268952824896399, 4.964870228720602, 0.0, 0.0, 0.0]), array('d', [-1.35, -1.05, -0.7500000000000001, -0.4500000000000002, -0.15000000000000013, 0.1499999999999999, 0.44999999999999973, 0.75, 1.0499999999999998, 1.3499999999999996, 1.65, 1.9499999999999997, 2.2499999999999996, 2.55, 2.85, 3.15, 3.4499999999999997, 3.7499999999999996, 4.049999999999999, 4.35]), array('d', [0.0, 0.0, 0.0, 0.0, 19.79365739344255, 0.0, 18.67219911843, 0.0, 20.4105774310009, 0.0, 0.0, 0.0, 0.0, 19.41768518529871, 20.598864576903765, 17.40978536289865, 21.015493233263065, 0.0, 0.0, 0.0]), array('d', [0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15])], 1: [array('d', [0.0, 0.0, 0.0, -5.295814838564534, -4.887042249228318, 4.82021934522815, 15.446903889742744, 16.1911

In [13]:
if False:
    for iz in x_arrays:
        arr = x_arrays[iz][0]
        arr_errors = x_arrays[iz][2]
        n = len(arr)
        # Find indices of valid (non -200) entries in each region
        valid_indices_low = [i for i, v in enumerate(arr) if v != -200 and i < 10]
        valid_indices_high = [i for i, v in enumerate(arr) if v != -200 and i >= 10]
        for i in range(n):
            if arr[i] == -200:
                if i < 10 and valid_indices_low:
                    closest = min(valid_indices_low, key=lambda x: abs(x - i))
                    arr[i] = arr[closest]
                    arr_errors[i] = arr_errors[closest]
                elif i >= 10 and valid_indices_high:
                    closest = min(valid_indices_high, key=lambda x: abs(x - i))
                    arr[i] = arr[closest]
                    arr_errors[i] = arr_errors[closest]

In [14]:
c3 = root.TCanvas("c3", "c3", 1600, 1600)
c3.Divide(2, 4)
graphs_x = []
graphs_y = []
for iz in range(len(z_means)):
    c3.cd(iz+1)
    if iz not in x_arrays: continue
    graph_x = root.TGraphErrors(len(x_arrays[iz][0]), x_arrays[iz][1], x_arrays[iz][0], x_arrays[iz][3], x_arrays[iz][2])
    graph_x.SetTitle(f"Z={get_zed_value(iz)}")
    graph_x.GetXaxis().SetTitle("Phi")
    graph_x.GetYaxis().SetTitle("DCA X/Y")
    graph_x.SetMarkerStyle(20)
    graph_x.SetMarkerColor(colors[iz])
    graphs_x.append(graph_x)
    graph_x.SetMaximum(50)
    graph_x.SetMinimum(-50)
    graph_x.Draw("AP")
c3.Draw()

In [15]:
if False:
    for iz in x_arrays:
        arr = x_arrays[iz][0]
        n = len(arr)
        # Find indices of valid (non -200) entries
        valid_indices = [i for i, v in enumerate(arr) if v != -200]
        for i in range(n):
            if arr[i] == -200:
                # Find closest valid index
                if valid_indices:
                    closest = min(valid_indices, key=lambda x: abs(x - i))
                    arr[i] = arr[closest]

In [16]:
if True:
    print(f"const double offsets_dca[{len(x_arrays)}][{len(x_arrays[0][0])}] =" + " {")
    for iz in range(len(x_arrays)):
        if iz > 0: print(",")
        print("    {", end="")
        for i in range(len(x_arrays[iz][0])):
            if i > 0: print(", ", end="")
            print(f"{x_arrays[iz][0][i]/10000:.3g}", end="")
        print("}", end="")
    print("\n};")

const double offsets_dca[8][20] = {
    {0, 0, 0, 0, -0.000279, 0, 0.00211, 0, 0.00088, 0, 0, 0, 0, 0.00131, 0.0007, 0.000163, 0.000496, 0, 0, 0},
    {0, 0, 0, -0.00053, -0.000489, 0.000482, 0.00154, 0.00162, 0.00116, 0, 0, 0, 0, 0.00107, 0.000351, -0.000269, -0.00129, 0.00131, 0, 0},
    {0, 0, 0, -0.00013, 0.000261, -0.0013, 0.000327, 0.000894, 0.000832, 0, 0, 0, 0, -0.00159, -0.000903, -0.000962, -0.000764, 0.00176, 0, 0},
    {0, 0, 0, -0.000309, -0.00021, -0.00162, -0.00129, -0.000485, -0.000711, 0, 0, 0, 0, -0.000713, -0.000833, -0.00157, -0.00231, 0.000402, 0, 0},
    {0, 0, 0, -0.000644, -0.00096, 5.43e-05, -0.00101, -0.000345, -7.39e-05, 0, 0, 0, 0, -0.0016, -0.000913, -0.0013, -0.000767, 0.000908, 0, 0},
    {0, 0, 0, -0.000636, -0.000673, 0.000945, -0.00185, -0.00111, -0.000724, 0, 0, 0, 0, -9.39e-05, -0.000194, -0.00121, -0.00209, 0.000219, 0, 0},
    {0, 0, 0, -0.000524, -0.000697, -0.000462, -0.00187, -0.00073, 0.000964, 0, 0, 0, 0, -0.000809, -0.000575, -0.0012, -0.0026

In [17]:
if True:
    print(f"const double offsets_dca_y[{len(x_arrays)}][2] =" + " {")
    for iz in range(len(x_arrays)):
        if iz > 0: print(",")
        print("    {"+f"{(x_arrays[iz][0][4]+x_arrays[iz][0][5])/10000:.3g}"+f", {x_arrays[iz][0][15]/10000:.3g}", end="")
        print("}", end="")
    print("\n};")

const double offsets_dca_y[8][2] = {
    {-0.000279, 0.000163},
    {-6.68e-06, -0.000269},
    {-0.00104, -0.000962},
    {-0.00183, -0.00157},
    {-0.000906, -0.0013},
    {0.000272, -0.00121},
    {-0.00116, -0.0012},
    {-0.00207, -0.00207}
};


## Formula for Alignment Fit

The function used for fitting the mean shift as a function of azimuthal angle (φ) and longitudinal position (z) is:

<div align="center">

Δφ(z, φ) = ([0] + [1]·z)/10000 · sin(φ) + ([2] + [3]·z)/10000 · cos(φ) + [4]/1000

</div>

**where:**
- [0], [1], [2], [3], [4] are fit parameters extracted from the data,
- z is the longitudinal coordinate (e.g., along the beam axis),
- φ is the azimuthal angle.

---

### Why use this function for a 3D histogram fit?

- **Physical Motivation:**  
  Misalignments of detector layers often manifest as sinusoidal modulations in the measured hit positions as a function of φ, with possible linear dependence on z due to tilts or twists.

- **Parameter Extraction:**  
  Fitting this function to the 3D histogram of mean residuals (mean shift vs. φ and z) allows extraction of alignment parameters:
    - The coefficients of sin(φ) and cos(φ) (and their z-dependence) correspond to translations and tilts of the detector layer in the transverse plane.
    - The constant term can represent a global offset.

- **Usage:**  
  After fitting, the obtained parameters can be used to correct (align) the detector geometry, improving tracking and vertexing performance.